In [47]:
# Import libraries and set desired options

from __future__ import division, print_function
# Disable Anaconda warnings
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns

import pickle
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from scipy.sparse import hstack
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.decomposition import PCA, TruncatedSVD
from tqdm import tqdm

##### Approaching the Problem
We will be solving the intruder detection problem analyzing his behavior on the Internet. It is a complicated and interesting problem combining the data analysis and behavioral psychology.

For example: Yandex solves the mailbox intruder detection problem based on the user's behavior patterns. In a nutshell, intruder's behaviour pattern might differ from the owner's one: 
- the breaker might not delete emails right after they are read, as the mailbox owner might do
- the intruder might mark emails and even move the cursor differently
- etc.

So the intruder could be detected and thrown out from the mailbox proposing the owner to be authentificated via SMS-code.
This pilot project is described in the Habrahabr article.

Similar things are being developed in Google Analytics and described in scientific researches. You can find more on this topic by searching "Traversal Pattern Mining" and "Sequential Pattern Mining".

In this competition we are going to solve a similar problem: our algorithm is supposed to analyze the sequence of websites consequently visited by a particular person and to predict whether this person is Alice or an intruder (someone else). As a metric we will use [ROC AUC](https://en.wikipedia.org/wiki/Receiver_operating_characteristic). We will reveal who Alice is at the end of the course.

### 1. Data Downloading and Transformation
Register on [Kaggle](www.kaggle.com), if you have not done it before.
Go to the competition [page](https://inclass.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2) and download the data.

First, read the training and test sets. Then explore the data and perform a couple of simple exercises:

In [171]:
# Read the training and test data sets
train_df = pd.read_csv('/Users/ilyas/Documents/Innopolis/AI_com/datasets/alice/train_sessions.csv',
                       index_col='session_id')
test_df = pd.read_csv('/Users/ilyas/Documents/Innopolis/AI_com/datasets/alice/test_sessions.csv',
                      index_col='session_id')

# Switch time1, ..., time10 columns to datetime type
times = ['time%s' % i for i in range(1, 11)]
train_df[times] = train_df[times].apply(pd.to_datetime)
test_df[times] = test_df[times].apply(pd.to_datetime)

# Sort the data by time
train_df = train_df.sort_values(by='time1')

# Look at the first rows of the training set
train_df.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,target
session_id,,,,,,,,,,,,,,,,,,,,,
21669,56,2013-01-12 08:05:57,55.0,2013-01-12 08:05:57,NaN,NaT,NaN,NaT,NaN,NaT,...,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,0
54843,56,2013-01-12 08:37:23,55.0,2013-01-12 08:37:23,56.0,2013-01-12 09:07:07,55.0,2013-01-12 09:07:09,NaN,NaT,...,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,0
77292,946,2013-01-12 08:50:13,946.0,2013-01-12 08:50:14,951.0,2013-01-12 08:50:15,946.0,2013-01-12 08:50:15,946.0,2013-01-12 08:50:16,...,2013-01-12 08:50:16,948.0,2013-01-12 08:50:16,784.0,2013-01-12 08:50:16,949.0,2013-01-12 08:50:17,946.0,2013-01-12 08:50:17,0
114021,945,2013-01-12 08:50:17,948.0,2013-01-12 08:50:17,949.0,2013-01-12 08:50:18,948.0,2013-01-12 08:50:18,945.0,2013-01-12 08:50:18,...,2013-01-12 08:50:18,947.0,2013-01-12 08:50:19,945.0,2013-01-12 08:50:19,946.0,2013-01-12 08:50:19,946.0,2013-01-12 08:50:20,0
146670,947,2013-01-12 08:50:20,950.0,2013-01-12 08:50:20,948.0,2013-01-12 08:50:20,947.0,2013-01-12 08:50:21,950.0,2013-01-12 08:50:21,...,2013-01-12 08:50:21,946.0,2013-01-12 08:50:21,951.0,2013-01-12 08:50:22,946.0,2013-01-12 08:50:22,947.0,2013-01-12 08:50:22,0


The training data set contains the following features:

- **site1** – id of the first visited website in the session
- **time1** – visiting time for the first website in the session
- ...
- **site10** – id of the tenth visited website in the session
- **time10** – visiting time for the tenth website in the session
- **target** – target variable, possesses value of 1 for Alice's sessions, and 0 for the other users' sessions
    
User sessions are chosen in the way they are not longer than half an hour or/and contain more than ten websites. I.e. a session is considered as ended either if a user has visited ten websites or if a session has lasted over thirty minutes.

There are some empty values in the table, it means that some sessions contain less than ten websites. Replace empty values with 0 and change columns types to integer. Also load the websites dictionary and check how it looks like:

In [172]:
# Change site1, ..., site10 columns type to integer and fill NA-values with zeros
sites = ['site%s' % i for i in range(1, 11)]
times = ['site%s' % i for i in range(1, 11)]
train_df[sites] = train_df[sites].fillna(0).astype('int')
test_df[sites] = test_df[sites].fillna(0).astype('int')
# Load websites dictionary
with open(r"/Users/ilyas/Documents/Innopolis/AI_com/datasets/alice//site_dic.pkl", "rb") as input_file:
    site_dict = pickle.load(input_file)

# Create dataframe for the dictionary
sites_dict = pd.DataFrame(list(site_dict.keys()), index=list(site_dict.values()), columns=['site'])
print(u'Websites total:', sites_dict.shape[0])
sites_dict.head()

Websites total: 48371


,site
25075,www.abmecatronique.com
13997,groups.live.com
42436,majeureliguefootball.wordpress.com
30911,cdt46.media.tourinsoft.eu
8104,www.hdwallpapers.eu


In [173]:
# Answer
print(test_df.shape, train_df.shape)

(82797, 20) (253561, 21)


In [174]:
# Our target variable
y_train = train_df['target']

# United dataframe of the initial data 
full_df = pd.concat([train_df.drop('target', axis=1), test_df])

# Index to split the training and test data sets
idx_split = train_df.shape[0]

For the very basic model, we will use only the visited websites in the session (but we will not take into account timestamp features). The point behind this data selection is: *Alice has her favorite sites, and the more often you see these sites in the session, the higher probability that this is an Alice's session, and vice versa.*

Let us prepare the data, we will take only features `site1, site2, ... , site10` from the whole dataframe. Keep in mind that the missing values are replaced with zero. Here is how the first rows of the dataframe look like:

In [175]:
# Dataframe with indices of visited websites in session
full_sites = full_df[sites]
full_sites.head()

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10
session_id,,,,,,,,,,
21669,56,55,0,0,0,0,0,0,0,0
54843,56,55,56,55,0,0,0,0,0,0
77292,946,946,951,946,946,945,948,784,949,946
114021,945,948,949,948,945,946,947,945,946,946
146670,947,950,948,947,950,952,946,951,946,947


Sessions are the sequences of website indices, and data in this representation is inconvenient for linear methods. According to our hypothesis (Alice has favorite websites) we need to transform this dataframe so each website has corresponding feature (column) and its value is equal to number of this website visits in the session. It can be done in two lines:

In [176]:
# sequence of indices
sites_flatten = full_sites.values.flatten()

# and the matrix we are looking for
full_sites_sparse = csr_matrix(([1] * sites_flatten.shape[0],
                                sites_flatten,
                                range(0, sites_flatten.shape[0]  + 10, 10)))[:, 1:]

In [177]:
full_sites_sparse.toarray().shape

(336358, 48371)

In [178]:
tf_idf = TfidfTransformer()
tf_idf_sparse = tf_idf.fit_transform(full_sites_sparse)
#pca = TruncatedSVD(n_components=100)
#pca.fit(tf_idf_sparse)
#tf_idf_sparse = pca.transform(tf_idf_sparse)

In [179]:
def calculate_add_features(dataset):
    dataset['unique_activities'] = np.count_nonzero(dataset[sites].values, axis=1)
    for i in range(1,10):
        dataset['timedelta'+str(i)] = (dataset['time'+str(i+1)] - dataset['time'+str(i)]) / pd.Timedelta('1 second')
    dataset['full_time'] = np.sum(dataset.iloc[:,-9:], axis=1)
    dataset['hour_start'] = np.array(np.floor(dataset['time1'].dt.hour), dtype=int)
    dataset['weekday'] = dataset['time1'].dt.weekday
    dataset = pd.concat([dataset, pd.get_dummies(dataset['weekday'])], axis=1)   
    dataset = pd.concat([dataset, pd.get_dummies(dataset['hour_start'])], axis=1)
    return dataset
full_df = calculate_add_features(full_df)

In [180]:
timedeltas = ['timedelta%s' % i for i in range(1,10)]
#full_df = full_df.drop(timedeltas, axis=1)
full_df[timedeltas] = full_df[timedeltas].fillna(0).astype(int)
#full_df = full_df.drop('weekday', axis=1)
#full_df = full_df.drop('hour_start', axis=1)

In [181]:
full_df.iloc[:5,-35:]

,timedelta2,timedelta3,timedelta4,timedelta5,timedelta6,timedelta7,timedelta8,timedelta9,full_time,hour_start,...,14,15,16,17,18,19,20,21,22,23
session_id,,,,,,,,,,,,,,,,,,,,,
21669,0,0,0,0,0,0,0,0,0.0,8,...,0,0,0,0,0,0,0,0,0,0
54843,1784,2,0,0,0,0,0,0,1786.0,8,...,0,0,0,0,0,0,0,0,0,0
77292,1,0,1,0,0,0,1,0,4.0,8,...,0,0,0,0,0,0,0,0,0,0
114021,1,0,0,0,1,0,0,1,3.0,8,...,0,0,0,0,0,0,0,0,0,0
146670,0,1,0,0,0,1,0,0,2.0,8,...,0,0,0,0,0,0,0,0,0,0


In [182]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
stdsk = MinMaxScaler()
#stdsk = StandardScaler()
added_feat = full_df.iloc[:, -35:].values
added_feat = stdsk.fit_transform(added_feat)
added_feat[1]

array([0.99111111, 0.00111173, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.99222222, 0.0625    ,
       0.83333333, 0.        , 0.        , 0.        , 0.        ,
       0.        , 1.        , 0.        , 0.        , 1.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ])

In [183]:
#new_feat_flatten = full_df.iloc[:, -3:].values.flatten()
#new_feat_sites_sparse = csr_matrix(([1] * new_feat_flatten.shape[0],
#                                new_feat_flatten,
#                                range(0, new_feat_flatten.shape[0]  + 3, 3)))[:, 1:]
tf_idf_add_features = csr_matrix(hstack([tf_idf_sparse, csr_matrix(added_feat)]))
tf_idf_add_features.toarray().shape

(336358, 48406)

### 3. Training the first model

So, we have an algorithm and data for it. Let us build our first model, using [logistic regression](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) implementation from ` Sklearn` with default parameters. We will use the first 90% of the data for training (the training data set is sorted by time), and the remaining 10% for validation. Let's write a simple function that returns the quality of the model and then train our first classifier:

In [204]:
from sklearn.model_selection import StratifiedKFold, cross_val_score, TimeSeriesSplit, GridSearchCV
def get_auc_lr_valid(X, y, C=0.7, seed=17, ratio = 0.9):
    # Split the data into the training and validation sets
    #idx = int(round(X.shape[0] * ratio))
    # Classifier training
    lr = LogisticRegression(C=C, random_state=seed, n_jobs=-1, class_weight='balanced')
    # Prediction for validation set
    #y_pred = lr.predict_proba(X[idx:, :])[:, 1]
    # Calculate the quality
    skf = TimeSeriesSplit(n_splits = 10)
    score = cross_val_score(lr, X, y, scoring='roc_auc', cv=skf, verbose=1)
    return score

In [205]:
%%time
# Select the training set from the united dataframe (where we have the answers)
X_train = tf_idf_add_features[:idx_split, :]
    
# Calculate metric on the validation set
print(get_auc_lr_valid(X_train, y_train))

[0.72263256 0.89341747 0.89214365 0.96627589 0.93046047 0.98176455
 0.90244479 0.95025183 0.83127818 0.97526833]
CPU times: user 20.8 s, sys: 492 ms, total: 21.2 s
Wall time: 21.5 s


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:   21.4s finished


[0.84244109 0.68597495 0.84858106 0.94721462 0.81296925 0.88563352
 0.91615432 0.84910028 0.9228053  0.90896216]

[0.81879    0.7120345  0.92653592 0.94933134 0.86531518 0.95170625
 0.90706718 0.89500456 0.96405043 0.96455432]
CPU times: user 20.2 s, sys: 626 ms, total: 20.8 s
Wall time: 21.7 s

[0.87778942 0.83542866 0.84947582 0.96646694 0.88261684 0.98011329
 0.8956218  0.92164434 0.77816609 0.97147264]
CPU times: user 14.4 s, sys: 447 ms, total: 14.8 s
Wall time: 15.4 s

In [202]:
skf = TimeSeriesSplit(n_splits = 10)
lr = LogisticRegression(random_state=42, n_jobs=-1,class_weight='balanced')
parameters = {'C': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]}
gcv = GridSearchCV(lr, parameters, n_jobs=-1, cv=skf, verbose=1, scoring='roc_auc')
gcv.fit(X_train, y_train)

Fitting 10 folds for each of 10 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   35.2s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  1.6min finished


GridSearchCV(cv=TimeSeriesSplit(max_train_size=None, n_splits=10),
       error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=-1, penalty='l2', random_state=42,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'C': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=1)

In [203]:
gcv.best_params_

{'C': 0.7}

The first model demonstrated the quality  of 0.91952 on the validation set. Let's take it as the first baseline and starting point. To make a prediction on the test data set ** we need to train the model again on the entire training data set ** (until this moment, our model used only part of the data for training), which will increase its generalizing ability:

In [43]:
# Function for writing predictions to a file
def write_to_submission_file(predicted_labels, out_file,
                             target='target', index_label="session_id"):
    predicted_df = pd.DataFrame(predicted_labels,
                                index = np.arange(1, predicted_labels.shape[0] + 1),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

In [206]:
# Train the model on the whole training data set
# Use random_state=17 for repeatability
# Parameter C=1 by default, but here we set it explicitly
st_lrn = int(0.2*X_train.shape[0])
lr = LogisticRegression(C=0.7, random_state=17, class_weight='balanced').fit(X_train, y_train)

# Make a prediction for test data set
X_test = tf_idf_add_features[idx_split:,:]
y_test = lr.predict_proba(X_test)[:, 1]

# Write it to the file which could be submitted
write_to_submission_file(y_test, 'alice_submission10.csv')